In [1]:
%reload_ext tutorial.tests.testsuite

# Functional Programming in Python

## Presentation slides
The following section contains the slides shown during the workshop

## Functional programming in python
Why functional programming?
- easy **reasoning*: *there are no surprises*
- suitable for **parallel** programming


## The principles of functional programming

- *Functions as values*
- *Pure functions*
- *Immutability of data*
- *Composition*
- *Referential  transparency*
- *type system*

## Functions as values

Functions are *objects* of the programming language.  They can be assigned and manipulated like other values:

```python
def x()-> int:
    return 3

y = x

print(x() == y())
```

This leads to the use of *higher-order-functions*, which take other functions as parameter. This idiom is useful to abstract over the behavior of programs.

For example, `map` is used to apply a function to a list of values:


In [31]:
from IPython.display import Markdown
import inspect
def my_fun(a: int)-> int:
    return a + 1

input_list = list(range(10))
result = list(map(my_fun, input_list))



Input: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], output: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
print(f"Input: {input_list}, output: {result}")

We can create *anonymous* functions using `lambda`

In [ ]:
new_fun = lambda x: x + 1

In [ ]:
new_fun(5)

## Pure functions

We prefer functions that have no *side-effects*: 

1. they do not modify the state of the program 
2. given the same input, they **always** return the same output.



This following function is not *pure* because it modifies the value of `x` every time it runs:


In [48]:
from typing import Any
x = [1,2,3]
def do_something(val: int) -> list[int]:
    return x.append(val)

def display_val(x: Any):
    print(f"{x=}, {id(x)=}")


x=[1, 2, 3], id(x)=140131238496640
x=[1, 2, 3, 1], id(x)=140131238496640
x=[1, 2, 3, 1, [...]], id(x)=140131238496640


In [ ]:
display_val(x)
do_something(1)
display_val(x)
do_something(x)
display_val(x)

`id(x)` returns the memory address of a variable. If it does not change, it means the variable refers to the same object. This is the case here, therefore we proved that `do_something` is not *pure*.

## Immutable Data

Whenever possible, we avoid *mutating* data, as mutation is a type of *side-effect*. 

When a variable is defined, its value **remains constant for the duration of the program execution**.

This makes it easier to reason about the flow of our program. 


For example, instead of inserting an element in an existing list as we did before, we create a new list by combining the old list with the new element:

In [36]:
x = [1, 2, 3]
print(f"{x=}")
y = x + [4,]
print("Appending")
print(f"{x=}")
print(f"{y=}")

x=[1, 2, 3]
Appending
x=[1, 2, 3]
y=[1, 2, 3, 4]


As you can see, the value of `x` does not change when we create `y`. This is in contrast to the previous example.

## List compherensions
Immutability of data in python is frequently expressed using *compherensions*.
These are for-like expressions that operate on iterables and produce *values*.

Let's look at an example of *mutation*: we want to produce a list of the first 10 squares:

In [60]:
a = []
for i in range(10):
    print(f"{a=}, {id(a)=}")
    a.append(i**2)

a=[], id(a)=140131238690688
a=[0], id(a)=140131238690688
a=[0, 1], id(a)=140131238690688
a=[0, 1, 4], id(a)=140131238690688
a=[0, 1, 4, 9], id(a)=140131238690688
a=[0, 1, 4, 9, 16], id(a)=140131238690688
a=[0, 1, 4, 9, 16, 25], id(a)=140131238690688
a=[0, 1, 4, 9, 16, 25, 36], id(a)=140131238690688
a=[0, 1, 4, 9, 16, 25, 36, 49], id(a)=140131238690688
a=[0, 1, 4, 9, 16, 25, 36, 49, 64], id(a)=140131238690688


`id(a)` returns the memory *address* of `a`. As you can see, we modify the contents of `a` over the for loop.

Let's rewrite this using a compherension and se what happens

In [63]:
a = [i for i in range(10)]
print(f"{a=}, {id(a)=}")
b = [i**2 for i in a]
print(f"{b=}, {id(b)=}")
print(f"{a=}, {id(a)=}")

a=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], id(a)=140131238601280
b=[0, 1, 4, 9, 16, 25, 36, 49, 64, 81], id(b)=140131238592768
a=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], id(a)=140131238601280


We see that the second expression produced a *new list* that we assigned  to `b`. The value of `a` remained unchanged.

## Referential transparency
This sounds scary, but it simply means that whenever we see a function call (or any expression), we can subsitute it with the value of the function.

Consider this example

In [69]:

def fun(x: int) -> int:
    return x + 2

y1 = fun(1) + 3


x_out = fun(1)
y2 = x_out + 3

print(f"{y1=}")
print(f"{y2=}")


y1=6
y2=6


We see that `x` is indeed referentially transparent. Now, look at another example:

In [68]:
import random
def x2() -> int:
    return random.randint(0, 10)

# This "captures" the expression x() + 3 without computing its value. We get the value by calling y1()
y1 = lambda : x2() + 3





In [ ]:
print(f"{y1()=}")
print(f"{y1()=}")

This is *not*. Every time we call `x2`, we get a different (random) value.

## Writeup

## References
Here are some additional references to help you in self-learning. Next to each link, we write if this is a video, text or another type of material

- The [functools reference](https://docs.python.org/3/library/functools.html) from the python standard library (text)
- The [itertools reference](https://docs.python.org/3/library/itertools.html) from the python standard library (text)
- [Functional programming howto](https://docs.python.org/3/library/functools.html) from the python documentation
- A [good, but old introduction](https://www.youtube.com/watch?v=Ta1bAMOMFOI) of functional programming (video)
- A [very high level](https://www.youtube.com/watch?v=Qa8IfEeBJqk) introduction of functional programming (video, advanced). Interesting, but not python-specific as it refers to Haskell
- A [general introduction on functional programming](https://www.youtube.com/watch?v=8z_bUIl_uPo). Very worth watching as it uses python for the examples (video)
- [Principles of functional programming](https://dev.to/jamesrweb/principles-of-functional-programming-4b7c)



## Introduction
Functional programming is an approach to programming where programs are built by composing and running functions that perform a series of transformation on data. This is in contrast with the common approach of *imperative programming*, where programs are written as series of statements which modify the *state* of the computation environment. Normally within functional programming, great focus is placed on *composability*, *immutability* and  *purity*. We are going to define these terms in more detail later.
## Why Functional Programming
Why do we choose functional programming? There are a series of advantages to this approach, namely:
- **Debugging** and **testing** are easy: there are no surprises because every function only does one thing and does not affect any other piece of the program.
- **Parallelisation** is trivial: because functions are just small boxes that take one input and produce an output and do not depend on other parts of the code in an implicit manner through global variables or other shared pieces of state, it is easy to make several functions run in parallel.
## The basic principles of functional programming
All modern programming languages have *functions* (or methods, procedures, subroutines, subprograms); these are groups of program statements that perform a certain computation. Functions are defined once for the whole program and can be reused at will throughout the program whenever we need to perform the specific computation they are defined for. Using functions, we can split our code in smaller units that are only responsible for a specific *functionality*; this helps us structuring our code in a clean and understandable form. 

The main principles we use in functional programming are:
- *purity*
- *immutability*
- *composability*
- *functions as values* (*higher order functions*)
- *referential transparency*

In the next few sections, we are going to briefly explore these concepts with some examples.


### Pure Functions (Purity)
In functional programming, we try to strive for *purity*, that is we want to define and use functions that only depend on their input, always return the same output for the same inputs and do not have any *side effects*, that is they do not indirectly affect any other part of our program. You can think of these functions as mathematical functions. 
Other examples of side effects are:
- printing to the program output
- reading or writing files
- generating and using random numbers
To besser understand this concept, let us look at the function `my_first_pure_function` we defined below:


In [1]:
def my_first_pure_function(x: int) -> int: 
    return x + 1

&#x1F600
Is this a pure function?
As you could probably imagine, this function is pure. Any time we run it, we get the same  result and running the function does not affect any other part of our program:

In [2]:
x = 1
print(f"x is {x}")
print(f"The result of calling the function is {my_first_pure_function(1)}")
print(f"x is {x}")
print(f"The result of calling the function is {my_first_pure_function(1)}")
print(f"x is {x}")

x is 1
The result of calling the function is 2
x is 1
The result of calling the function is 2
x is 1


As a rule of thumb, any function that does not modify variables outside of its *scope* and only uses mathematical functions is a pure function.
Now, consider this function instead:

In [3]:
x = ["short", "list"] 
def do_something(y: str) -> None:
    x.append(y)

> &#x1F600 Is this function pure? 

Let's try and run it. 

In [4]:
print(x)
do_something("a")
print(x)

['short', 'list']
['short', 'list', 'a']


As you see from the output, the function modified the list `x`. Therefore, this function is not pure. This leads us to the next principle, **immutability**.

### Immutability
When writing programs in functional style, we usually avoid functions like `do_something`. Instead of modifying existing data (*mutation*), you write functions that transform your data and return new objects.
In the case of the function above, we would rewrite it as follows:


In [5]:
x = ["short", "list"] 
def do_something_immutable(x: "list[str]", y: str) -> "list[str]":
    return x + [y]

print(x)
print(do_something_immutable(x, "a"))
print(x)

TypeError: 'type' object is not subscriptable

The output shows that `do_something_immutable` does not change `x`. It also does not reference to `x` outside of the scope but takes it as an argument. Instead of modfying the original `x` list, it returns a new list. When we want to keep immutability, this is the style we work with. If we adopt this style, it is easier to reason about the flow of our program as there are no variable that are being modified *at distance* by other function calls.

### Composition
Another important aspect of functional programming is `composition`. This means that whenever we want to perform multiple operations, we avoid writing intermediate variables in the global state. Instead, we design our functions in such a way that one functions output can be directly fed into the next function.


### Higher Order Functions / Functions as Values
Another important principle of functional programming is that **functions are values**. In programming languages (like python) that support a functional style, we can manipulate functions with the language, we can pass them around in a variable and even use functions as parameters for another function.

As an example, consider the function `function_caller`:



In [ ]:
from typing import Callable, Any
def function_caller(f: Callable[[Any], Any], arg: Any) -> Any:
    print(f"Calling the function {f} with argument {arg}")
    res = f(arg)
    print(f"The result of the call was {res}")
    return res

This function takes a function `f` and its argument `args` as input and return the result of calling the function, while additionally printing a message on the standard output of the program. Let's try this out.
To do so, we define a new function `add_five`:

In [ ]:
def add_five(x: int) -> int:
    return x + 5

Now we can call `function_caller` with `add_five` and another number as arguments:

In [ ]:
function_caller(add_five, 4)

Calling the function <function add_five at 0x7fb68848f1c0> with argument 4
The result of the call was 9


9

This example was a bit convoluted, but it shows that in python we can use functions as values and even pass them as arguments to other functions.

This is actually useful in many cases; a typical example being numerical optimization, where we want to find the parameters to a function that minimize a certain criteria.

Other than these specific use cases, there are some common *higher order functions*  (HoF), or functions that take functions as parameters that are common in most programming languages.  We will look at a few in the next section.



### Referential transparency
The last principle we like to discuss is (*referential transparency*)[https://en.wikipedia.org/wiki/Referential_transparency]. This complex-sounding name simply means that we can replace any expression with its value without changing the behavior of the program.
Consider this program:

In [100]:
def x() -> int:
    return 5

y1 = x() + 5
y2 = x() + 5

x() + 5 == y1

y1 == y2


If `x` is referentially transparent, we can replace its value in the expression `y1 = x() + 5` and we obtain the same value. In other words, the value of `y` does not change if we execute it again, as shown in the code above: we have two expressions `y1` and `y2` that use the same code and because `x` is referentially transparent, 
their value is equal.

In this case, this is true because the value of `x()` is simply 5 and is invariant.

However, consider this counterexample:


In [101]:
import random
def x() -> int:
    return random.randint(0, 10)


y1 = x() + 5

y2 = x() + 5

y1 == y2

AssertionError: 

in this case, because the function returns a different random number every time we call it, the expression `y1` has a different *value* compared to `y2` even though they both have the same *form*.



## Mapping / Iteration 

In this section, we will look at the application of some of the principles stated above in python. An important application of functional programming in python is the manipulation of iterables and lists. 

### Using map and filter

Python offers the `map` function which can be used to apply a given function to all members of an `iterable`. Do not worry too much about what an `iterable` exactly is, just see it as a generalisation of a list, representing all objects that we can `iterate` over, accessing their values one by one. Let's see an example of `map` using the `add_five` function:

In [ ]:
map(add_five, [1,2,3,4])

The output is a bit confusing! The reason is that `map` does not return a list but a [map](https://docs.python.org/3/library/functions.html#map) object. This object is `lazy`, that means that the values are only generated when we access them, for example by iterating over the `map` object. 
Fortunately, we can easily convert this into a list by calling the `list` function:

In [ ]:
list(map(add_five, [1,2,3,4]))

[6, 7, 8, 9]

This is equivalen to using a for loop:


In [ ]:
a = []
for i in [1,2,3,4]:
    a.append(add_five(i))
print(a)

[6, 7, 8, 9]


However, as you can see this former style is not very functional because we mutate an existing list `a` by appending the results of calling `add_five` to it. We will see later that python offers a syntax called *list compehrension* to express this operation in a more functional style.
`map` is very useful when we want to apply the same functions to a list of parameters and we do not want to write a `for` loop. It makes for more coincise code, while sacrificing some readability. 

#### Filtering
Another basic higher order function is `filter`. As the name says, this function is used to filter an `iterable` using a *predicate function*. This is a function that takes a value and return `true` or `false`. With `true`, the current element is kept, with `false` it is discarded.
We now try to write a predicate function that only keeps even numbers.

<a name="filter-example"></a>

In [ ]:
def is_even(x: int) -> bool:
    return x % 2 == 0

Now we are ready to try `filter` using `is_even`. Because `filter` returns a `filter` object, we wrap it in `list` to directly see the result as a list:

In [ ]:
list(filter(is_even, [1,2,3,4,5]))

[2, 4]

#### Reducing
A third basic HoF is *reduction*. This is a function that takes a function `f(x, y)` of two arguments and an iterable `l` an applies the function  to every element in the iterable *cumulatively* to produce one value. It works in the following way:
- The first argument of `f`, `x` is the current value of the accumulation. At first, this corresponds to the first element of `l`
- The second argument of `f`. `y` is the current element of the iterable. At first, this corresponds to the second element of `l`

Because of this behavior, this function is useful to compute sums or similar aggregation over a list of numbers. In python, this function is available in the `functools` [package](https://docs.python.org/3/library/functools.html) as a part of the python standard library.

As an example of using `reduce`, consider the following snippet:


In [ ]:
import functools


def spy(x: int, y: int) -> int:
    print(f"x: {x}, y: {y}")
    return x * y

result = functools.reduce(spy, [1, 2, 3, 4, 5])
print(result)

x: 1, y: 2
x: 2, y: 3
x: 6, y: 4
x: 24, y: 5
120


As you can see, we are computing the cumulative product of `[1, 2, 3, 4, 5]`, that is the *factorial* of 5, or `5!`

### List Compehrensions
Many of the operations in the previous section can be performed in a different (some would say more *pythonic*) way using [list compehrensions](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).
These look like mini for-loops inside square brackets and are very useful to write programs in a functional style while keeping code more readable than using higher order functions.  

For example, if we want to double all integers between 1 and 10 and store the result in a list,  we can proceed in the classical imperative way:

In [ ]:
doubles = []

for i in range(10):
    doubles.append(i * 2)

print(f"The doubles are {doubles}")


The doubles are [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


Or we can use `map` like this:

In [ ]:
doubles = map(lambda x: x * 2, range(10))

print(list(doubles))


[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


Again here we wrapped `doubles` in `list` because `map` is lazy and does not return a list immediately but a `map` object which can be converted into a list using `list`. 
This solution is more elegant and avoids side effects because we do not modify the previously defined `doubles` list, instead we directly produce a new list with the desired result.

Now, we solve the same problem using a list compherension:

In [ ]:
doubles = [i * 2 for i in range(10)]
print(doubles)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


This looks a lot like the solution above for the for loop, but it is an *expression* instead of a *statement*. [Expressions](https://www.baeldung.com/cs/expression-vs-statement) in programming are pieces of code that return a value and hence can be composed together to form more complex expressions and used anywhere a value is needed. In functional style, we prefer working with expressions instead of statements because we try to keep [*referential transparency*](https://en.wikipedia.org/wiki/Referential_transparency).

Compherensions can do more than just iterate over a list. We can use them to filter elements of a list by adding an `if` statement at the end.
For example, consider the example of filtering even numbers  from a list [above](#filter-example); we can rewrite this with list compherensions like this:


In [ ]:
def is_even(x: int) -> bool:
    return x % 2 == 0

[i for i in range(5) if is_even(i)]

[0, 2, 4]

We can also combine (*nest*) list compherensions. This is equivalent to a nested for loop. For example consider this expression:

In [6]:
[(i,j) for i in range(5) for j in range(5)]

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4)]

This returns all the combination of numbers between 0 and 4 as a list of pairs. In the first part of the compherension, we can use any expression instead of just referencing the variable. For example we can return a description of the pair `i,j` in text:

In [9]:
[f"The current pair is {(i,j)}" for i in range(5) for j in range(5)]

['The current pair is (0, 0)',
 'The current pair is (0, 1)',
 'The current pair is (0, 2)',
 'The current pair is (0, 3)',
 'The current pair is (0, 4)',
 'The current pair is (1, 0)',
 'The current pair is (1, 1)',
 'The current pair is (1, 2)',
 'The current pair is (1, 3)',
 'The current pair is (1, 4)',
 'The current pair is (2, 0)',
 'The current pair is (2, 1)',
 'The current pair is (2, 2)',
 'The current pair is (2, 3)',
 'The current pair is (2, 4)',
 'The current pair is (3, 0)',
 'The current pair is (3, 1)',
 'The current pair is (3, 2)',
 'The current pair is (3, 3)',
 'The current pair is (3, 4)',
 'The current pair is (4, 0)',
 'The current pair is (4, 1)',
 'The current pair is (4, 2)',
 'The current pair is (4, 3)',
 'The current pair is (4, 4)']

### Combinining and splitting iterators
Sometimes, we need to iterate over multiple lists in a special fashion, for example over two lists in parallel or we want to compute all the combinations of elements of two lists etc.
We can solve many ot these problems with the use of list compherensions, but sometimes there are more elegant solutions by using the [`itertools`](https://docs.python.org/3/library/itertools.html) package, a part of the python standard library.

Lets see some example of problems we can solve by using these tools.

- Iterating over two lists in parallel: 
    consider the two lists `numbers = [1, 2, 3, 4, 5]` and `words = ["one", "two", "three" , "four", "five"]`. We want to produce a list of strings consisting of the content of `numbers` and `words`, such that each number in `numbers` corresponds to the right word in `words`. We can do this by using the `zip` buil-in function:
`zip` takes a number of `iterables` as an input and returns a list of tuples. The i-th element of this list is a tuple with the i-th element of each iterable, proceeding until the shortes iterable is exhausted. Using `zip`, we solve our problem like this:

In [11]:
numbers = [1, 2, 3, 4, 5]
words = ["one", "two", "three" , "four", "five"]
[f"The number {i} is called {j}" for i, j in zip(numbers, words)]

['The number 1 is called one',
 'The number 2 is called two',
 'The number 3 is called three',
 'The number 4 is called four',
 'The number 5 is called five']

-  Producing permutations of elements.
    Suppose we have a list containing the letters `["A", "B", "C", "D", "E"]` and we want to produce all four letter words from them. We can use   `itertools.permutations`. Once again, we wrap the operation in `list` to obtain a list as output:

In [13]:
import itertools
letters = ["A", "B", "C", "D", "E"]
list(itertools.permutations(letters, 4))

[('A', 'B', 'C', 'D'),
 ('A', 'B', 'C', 'E'),
 ('A', 'B', 'D', 'C'),
 ('A', 'B', 'D', 'E'),
 ('A', 'B', 'E', 'C'),
 ('A', 'B', 'E', 'D'),
 ('A', 'C', 'B', 'D'),
 ('A', 'C', 'B', 'E'),
 ('A', 'C', 'D', 'B'),
 ('A', 'C', 'D', 'E'),
 ('A', 'C', 'E', 'B'),
 ('A', 'C', 'E', 'D'),
 ('A', 'D', 'B', 'C'),
 ('A', 'D', 'B', 'E'),
 ('A', 'D', 'C', 'B'),
 ('A', 'D', 'C', 'E'),
 ('A', 'D', 'E', 'B'),
 ('A', 'D', 'E', 'C'),
 ('A', 'E', 'B', 'C'),
 ('A', 'E', 'B', 'D'),
 ('A', 'E', 'C', 'B'),
 ('A', 'E', 'C', 'D'),
 ('A', 'E', 'D', 'B'),
 ('A', 'E', 'D', 'C'),
 ('B', 'A', 'C', 'D'),
 ('B', 'A', 'C', 'E'),
 ('B', 'A', 'D', 'C'),
 ('B', 'A', 'D', 'E'),
 ('B', 'A', 'E', 'C'),
 ('B', 'A', 'E', 'D'),
 ('B', 'C', 'A', 'D'),
 ('B', 'C', 'A', 'E'),
 ('B', 'C', 'D', 'A'),
 ('B', 'C', 'D', 'E'),
 ('B', 'C', 'E', 'A'),
 ('B', 'C', 'E', 'D'),
 ('B', 'D', 'A', 'C'),
 ('B', 'D', 'A', 'E'),
 ('B', 'D', 'C', 'A'),
 ('B', 'D', 'C', 'E'),
 ('B', 'D', 'E', 'A'),
 ('B', 'D', 'E', 'C'),
 ('B', 'E', 'A', 'C'),
 ('B', 'E',

Another useful trick to be aware of when working with lists is [`unpacking`](https://docs.python.org/3/tutorial/controlflow.html#tut-unpacking-arguments). In python, we can extract elements from a list using the assignment statement.  For example, if we have a two-element list, we can write

In [64]:
a, b = [1, 2]

and extract the two elements in the variables `a` and `b`. If you try the same statement with a three-element list you will get an error:

In [65]:
try:
    a, b = [1, 2, 3]
except ValueError:
    print("It does not work")

ValueError: too many values to unpack (expected 2)

However, you can extract *all remaining elements* with the star notation:

In [17]:
a, *b = [1, 2, 3]

If we use `*` before an argument in a function call, this will unpack the argument and pass each component to the function separately, as you can see in this example:

In [23]:
def print_variables(a: any, b: any, c: any):
    print(f"a is {a}, b is {b}, c is {c} ")

print_variables(*[1, 2, 3])
print_variables(1, 2, 3)
print_variables(*[[1, 2, 3], [4, 5, 6], [5, 6, 7]])

a is 1, b is 2, c is 3 
a is 1, b is 2, c is 3 
a is [1, 2, 3], b is [4, 5, 6], c is [5, 6, 7] 


Be careful, this only works if the list/iterable you unpack is as long as the arguments you want to pass to your function.



## Exercises


### Pure or impure functions? 🌶️

For each of the functions below, determine whether they are pure or impure

In [6]:
def fun1(a: "list[str]") -> None:
    a.append("b")


def fun2(a: int) -> int:
    return a + 2


def fun3(a: "dict[str, str]"):
    a["test"] = "dest"



### Keeping only multiples of n 🌶️

Given a list `L` of integers, write a function that only keeps the numbers that are multiples of a given constant `k`.

- Example 1: given `nums = [1, 2, 3, 4, 5]`, and `k = 2`, the result must be `[2, 4]`

- Example 2: given `nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] ` and `k = 5`, the result is `[5, 10]`

Write your function in the cell below inside of the `solution_exercise1` function. The function receives a list `l` as an input and should return another list

In [89]:
%reload_ext tutorial.tests.testsuite


In [90]:
def solution_exercise1(l: "list[int]"):
    return [1, 2, 3]

<module '__main__'>

['In', 'Out', '_', '_24', '_26', '_27', '_28', '_29', '_30', '_32', '_45', '_49', '_50', '_51', '_53', '_54', '_56', '_57', '_58', '_62', '_63', '_64', '_66', '_67', '_68', '_69', '_70', '_73', '_74', '_75', '_76', '_77', '_78', '_79', '_81', '_83', '_84', '_85', '_86', '_87', '_88', '_90', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '__vsc_ipynb_file__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_i29', '_i3', '_i30', '_i31', '_i32', '_i33', '_i34', '_i35', '_i36', '_i37', '_i38', '_i39', '_i4', '_i40', '_i41', '_i42', '_i43', '_i44', '_i45', '_i46', '_i47', '_i48', '_i49', '_i5', '_i50', '_i51', '_i52', '_i53', '_i54', '_i55', '_i56', '_i57', '_i58', '_i59', '_i6', '_i60', '_i61', '_i62', '_i63', '_i64', '_i65', '_i66', '_i67', '_i68', '_i69', '_i7', '_i70', '_i71', '_i72', '_i73', '_i7

ERROR: file or directory not found: tutorial/tests/test_<module '__main__'>.py::test_exercise1



### Transposing a Matrix 🌶️🌶️

Consider a matrix `M` represented row-wise as a list of lists `[[1, 2, 3], [4, 5, 6], [7, 8, 8]]`.
Write a function `transpose` that returns the transpose of `M`, the matrix obtained by exchanging rows and columns

- Example 1: given `M=[[1, 0], [0, 1]]`, the result must be `[[1, 0], [0, 1]]`
- Example 2: given `M=[[1, 2, 3], [4, 5, 6], [7, 8, 8]]` the result must be `[1, 4, 7], [2, 5, 8], [3, 6, 8]]`

In [14]:
def transpose(m: "list[list[int]]") -> "list[list[int]]":
    return list(zip(*m))

M=[[1, 2, 3], [4, 5, 6], [7, 8, 8]]
transpose(M)

[(1, 4, 7), (2, 5, 8), (3, 6, 8)]

### Flattening list of lists 🌶️🌶️

Imagine we receive a list of lists `L` like `[[1, 2], [3, 4]]`. Write a function `flatten` that converts this list into a `flat` list like `[1, 2, 3, 4]`. 


In [10]:
print(functools.reduce(lambda x,y: x + y, [[1,2,3], [1,2]]))

[1, 2, 3, 1, 2]


### Counting initials 🌶️🌶️🌶️

Given a list `W` of english words, write a function `count_initial_frequency` that counts how many words begin with each letter of the english alphabet and returns the result as a list of tuples (letter, count).

You can get a list of words (as a single string separated by newlines `\n` from [here](https://www.mit.edu/~ecprice/wordlist.10000)). The code below shows how to download this string in a variable called `words`. 

In [53]:
import requests
words = requests.get("https://www.mit.edu/~ecprice/wordlist.10000").text
words_list = words.splitlines()





In [57]:
grouped = [(k, len(list(v))) for k, v in itertools.groupby(sorted(words_list, key = lambda x: x[0]), key = lambda x: x[0])]

print(grouped)

[('a', 720), ('b', 536), ('c', 1015), ('d', 560), ('e', 466), ('f', 432), ('g', 288), ('h', 334), ('i', 386), ('j', 127), ('k', 98), ('l', 372), ('m', 555), ('n', 231), ('o', 226), ('p', 792), ('q', 46), ('r', 581), ('s', 1005), ('t', 549), ('u', 127), ('v', 190), ('w', 288), ('x', 12), ('y', 42), ('z', 22)]


### Counting initials frequency 🌶️🌶️🌶️
If you could solve the previous exercise, you now have a list `I` of the form `[('a', 20), ('b', 30), ...]`

Write a function `frequency` that computes the *relative frequency* of each letter in the list. 

In [60]:
def frequency(l: "list[(str, int)]") -> "list[(str, int)]":
    total = sum(map(lambda x: x[1], l))
    return [(letter, freq/total) for letter, freq in l]
frequency(grouped)

[('a', 0.072),
 ('b', 0.0536),
 ('c', 0.1015),
 ('d', 0.056),
 ('e', 0.0466),
 ('f', 0.0432),
 ('g', 0.0288),
 ('h', 0.0334),
 ('i', 0.0386),
 ('j', 0.0127),
 ('k', 0.0098),
 ('l', 0.0372),
 ('m', 0.0555),
 ('n', 0.0231),
 ('o', 0.0226),
 ('p', 0.0792),
 ('q', 0.0046),
 ('r', 0.0581),
 ('s', 0.1005),
 ('t', 0.0549),
 ('u', 0.0127),
 ('v', 0.019),
 ('w', 0.0288),
 ('x', 0.0012),
 ('y', 0.0042),
 ('z', 0.0022)]

### Finding palindromes 🌶️🌶️🌶️
Consider again the `words` string, containing all the words separated by `\n`. Write a function `palindromes` that returns the list of all *palindromes*. A *palindrome* is a word that reads the same left-to-right and right-to-left;
for example:
- rotator
- wow
- noon

In [63]:
list(filter(lambda x: x == x[::-1], words_list))

['a',
 'aa',
 'aaa',
 'ada',
 'aka',
 'ala',
 'ana',
 'anna',
 'ata',
 'b',
 'bb',
 'bob',
 'boob',
 'c',
 'cc',
 'civic',
 'd',
 'dad',
 'dd',
 'did',
 'dod',
 'dvd',
 'e',
 'ee',
 'eve',
 'eye',
 'f',
 'ff',
 'g',
 'gg',
 'gig',
 'h',
 'hh',
 'i',
 'ii',
 'iii',
 'j',
 'jj',
 'k',
 'l',
 'level',
 'lil',
 'll',
 'lol',
 'm',
 'mem',
 'mm',
 'mom',
 'n',
 'nn',
 'non',
 'noon',
 'o',
 'oo',
 'ooo',
 'p',
 'pgp',
 'php',
 'pop',
 'pp',
 'psp',
 'q',
 'r',
 'radar',
 'refer',
 'rr',
 's',
 'sas',
 'sees',
 'sms',
 'ss',
 'stats',
 'sys',
 't',
 'tft',
 'tit',
 'tt',
 'u',
 'v',
 'w',
 'wow',
 'ww',
 'www',
 'x',
 'xanax',
 'xx',
 'xxx',
 'y',
 'z']